<a href="https://colab.research.google.com/github/Cipe96/EEG-Recognition/blob/main/Classificatore_Ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**EEG Recognition: Classificatore Ensamble**</font>
</br><font size=3>*Marco Cipollina, Riccardo Era*</font>


<p style="font-size:4px;" align="justify">Dopo avere addestrato il modello convolutivo costruito per questo progetto sia sui dati della run 01 (basale, basale occhi aperti), sia run 02 (basale, occhi chiusi) che sui dati della run 06 (immaginare di chiedere pugni e piedi in determinati momenti) , in questo notebook vogliamo analizzare le capacità di un classificatore Ensamble (formato dalla media della somma pesata per classe dei tre classificatori) di classificare i dati di una qualsiasi altra run.</p>

<p style="font-size:4px;" align="justify">....</p>

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os

SEED = 96                                                                       #Impostiamo un seme specifico per garantire la replicabilità degli esperimenti

In [ ]:
%%capture
drive.mount('/content/drive', force_remount=True)
# elimina la cartella sample_data creata automaticamente
! rm -r /content/sample_data

In [ ]:
#@title Percorso della cartella del progetto su Google Drive:

#@markdown Se la cartella del progetto si trova nella root di Drive, scrivere solo il suo nome:

PERCORSO_DRIVE = "EEG Recognition" #@param {type:"string"}

PERCORSO_DRIVE = '/content/drive/MyDrive/' + PERCORSO_DRIVE

In [ ]:
sys.path.append(PERCORSO_DRIVE)                                                 # ci permetterà di importare le funzioni presenti in altri file
from shared_utilities import convolutivo                                        # importare il classificatore convolutivo

<a name="2"></a>
# **Download**

In [704]:
#@title Seleziona la Run che vuoi utilizzare (fra quelle che hai sul drive):
#@markdown Inserisci un valore compreso fra 1 e 14 inclusi:

run = 6 #@param {type:"integer"}
run = f"R{run:02d}"


In [705]:
#@title Seleziona la banda che vuoi utilizzare:
#@markdown (delta, theta, alpha, beta, gamma, broadband, personalizzata, ABG)

banda = 'ABG' #@param ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broadband', 'personalizzata', 'ABG']
save_path = f'{PERCORSO_DRIVE}/Dati preprocessati/{run}/{banda}'

In [706]:
# Carica i dati e le etichette del test set
test_data = np.load(f"{save_path}/test_data_{banda}.npy")
test_labels = np.load(f"{save_path}/test_labels_{banda}.npy")

Controlliamo che la shape sia coerente con quella che ci servirà per il modello di classificazione

In [707]:
# Verifica le dimensioni dei dataset e delle etichette
print(f"Test set shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

Test set shape: (1417, 240, 64), Test labels shape: (1417,)


Effettuiamo uno shuffle del dataset per evitare di dare in pasto i dati al modello in maniera ordinata, permettendo così un addestramento meno condizionato.

In [708]:
#shuffle dei dataset e ettichette con stesso ordine di shuffle
test_data, test_labels = shuffle(test_data, test_labels, random_state=SEED)


<a name="3"></a>
# **Convolutivo addestrato su Run 01**

In [709]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R01/Pesi convolutivo/'
model01 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [710]:
model01.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [711]:
predizioni01 = model01.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step


<a name="3"></a>
# **Convolutivo addestrato su Run 02**

In [712]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R02/Pesi convolutivo/'
model02 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [713]:
model02.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_2 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_5           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 01

In [714]:
predizioni02 = model02.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step


<a name="3"></a>
# **Convolutivo addestrato su Run 06**

In [715]:
#@title Carica il modello (architettura, pesi e configurazioni)
weights_path = f'{PERCORSO_DRIVE}/Dati preprocessati/R06/Pesi convolutivo/'
model06 = tf.keras.models.load_model(weights_path + f'best-{banda}.keras')
print("architettura, pesi e configurazione caricati con successo")


architettura, pesi e configurazione caricati con successo


Verifichiamo che il modello sia costruito come vogliamo.

In [716]:
model06.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_1 (SeparableConv1D) │ (None, 240, 64)             │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 240, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 240, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 109)                 │           7,085 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,825 (136.04 KB)

 Trainable params: 11,565 (45.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 23,132 (90.36 KB)

Preleviamo le probabilità per ogni campione dal modello addestrato su Run 06

In [717]:
predizioni06 = model06.predict(test_data)

45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 148ms/step


<a name="3"></a>
# **Classificatore Ensamble**

Qui di seguito è possibile combinare come si vuole il classificatore Ensamble.
In caso di combinazione a due, i classificatori avranno voto pesato in egual modo, sommando le loro predizioni per classe.
Generalmente abbiamo ottenuto i risultati migliori sommando le predizioni di tutti e tre i classificatori dando peso doppio al terzo (addestrato sul doppio dei dati).

In [730]:
#@title Combina il tuo Ensamble utilizzando i classificatori addestrati rispettivamente sulle Run 01, 02 e 06:

modello_01 =  True #@param {type:"boolean"}
modello_02 =  True #@param {type:"boolean"}
modello_06 =  False #@param {type:"boolean"}

In [575]:
#@title Matrice per racchiudere tutte gli esperimenti fatti : attenzione a non resettarla

# Definiamo le combinazioni di modelli
combinazioni_modelli = [
    'modello_01',
    'modello_02',
    'modello_06',
    'modello_01 + modello_02',
    'modello_01 + modello_06',
    'modello_02 + modello_06',
    'modello_01 + modello_02 + modello_06'
]

# Definiamo i test set (Run 01, Run 02, ...)
test_sets = ['R01', 'R02', 'R03', 'R04', 'R05', 'R06']

# Creiamo una matrice vuota per memorizzare i risultati
# La matrice avrà dimensioni: (numero combinazioni modelli) x (numero test sets)
matrice_resultati = np.zeros((len(combinazioni_modelli), len(test_sets)))

# Creiamo il DataFrame per organizzare i dati in modo leggibile
matrice_df = pd.DataFrame(matrice_resultati, index=combinazioni_modelli, columns=test_sets)

In [731]:
#@title Combina i voti e calcola l'accuratezza dell'Ensamble
combinazione = ""
if modello_01 & modello_02 & modello_06:
  predizioni = (predizioni01 + predizioni02 + 2*predizioni06) / 4
  combinazione = "modello_01 + modello_02 + modello_06"
elif modello_01 & modello_02:
  predizioni = (predizioni01 + predizioni02) / 2
  combinazione = "modello_01 + modello_02"
elif modello_01 & modello_06:
  predizioni = (predizioni01 + predizioni06) / 2
  combinazione = "modello_01 + modello_06"
elif modello_02 & modello_06:
  predizioni = (predizioni02 + predizioni06) / 2
  combinazione = "modello_02 + modello_06"
elif modello_01:
  predizioni = predizioni01
  combinazione = "modello_01"
elif modello_02:
  predizioni = predizioni02
  combinazione = "modello_02"
elif modello_06:
  predizioni = predizioni06
  combinazione = "modello_06"
else:
  print("Nessun modello selezionato")
predizioni_finali = np.argmax(predizioni, axis=1)

#Calcola l'accuratezza confrontando le predizioni finali con le etichette
accuratezza = accuracy_score(test_labels, predizioni_finali)
print(f"Accuratezza dell'ensemble: {accuratezza:.2%}")

matrice_df.loc[combinazione, run] = float(f"{accuratezza*100:.2f}")
print(matrice_df)

Accuratezza dell'ensemble: 61.61%
                                        R01    R02    R03    R04    R05    R06
modello_01                            99.24  52.29  68.42  65.91  60.33  54.48
modello_02                            66.51  97.86  47.12  50.32  42.73  45.52
modello_06                            81.65  54.59  87.83  93.37  90.77  99.72
modello_01 + modello_02               98.93  97.55  71.31  67.82  64.09  61.61
modello_01 + modello_06               98.78  63.30  89.73  94.07  90.42  98.87
modello_02 + modello_06               90.83  96.33  89.17  93.01  90.42  99.29
modello_01 + modello_02 + modello_06  95.87  81.19  90.86  94.57  91.98  99.51


In [732]:
#@title Per salvare la matrice qui su Colab (da salvare poi manualmente)
matrice_df.to_csv('matrice_accuratezze_sep.csv', index=True)
# Salva solo i dati numerici del DataFrame in formato .npy
np.save('matrice_accuratezze_sep.npy', matrice_df.values)